In [1]:
import requests
import json
from pathlib import Path

# 1. Define Functions & Parameters

In [2]:
OLLAMA_API_URL = "http://localhost:11434/api/chat"

In [3]:
def initial_setup():
    instruction = 'You are a friendly and helpful data analyst. The user will ask you question about data analysis, be prepared.'
    chat= [{'role':'system',
            'content':instruction}]
    payload = {
    "model":"deepseek-r1:8b",
    "messages":chat,
    "stream":False,
    "temperature": 0.0
}
    response = requests.post(OLLAMA_API_URL,json=payload).json()

    chat.append({'role':'assistant',
                        'content':response['message']['content']})
    return chat

In [4]:
def generate_sql_query(natural_language):
    # Get the current directory of the script
    script_dir = Path.cwd()

    # Navigate to the parent directory and then into the Context folder
    context_dir = script_dir.parent / "Context"

    # Construct the full path to the file
    file_path = context_dir / "MASTER_FUNDING.txt"

    # Open context file in read mode
    with open(file_path, "r") as file:
        # Read the entire file content
        content = file.read()

    prompt = f"""
You are an expert SQL generator. Translate the following natural language request into a valid SQL query. 
Make sure the query is properly formatted and is tailored for our database schema.
The table in the database is master_funding, it uses the following schema:
{content}

Request: {natural_language}

SQL Query:
"""
    payload = {
        "model": "deepseek-r1:8b",  # if needed, or remove if the API defaults to it
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0,  # Lower temperature can help ensure deterministic output
        "stream":False
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response_data = response.json()
    
    # The structure may vary based on the API; assume generated text is in this field:
    sql_query = response_data.get("generated_text", "").strip()
    return sql_query

In [5]:
def generate_insight(natural_language, sql_query, query_result):
    # Format the query result in a readable way (e.g., JSON)
    formatted_result = json.dumps(query_result, indent=2)
    
    prompt = f"""
You are a data analyst. I first received the following natural language request:
"{natural_language}"

Based on that, an SQL query was generated and executed:
{sql_query}

The query returned the following result:
{formatted_result}

Please provide a detailed explanation of the insights, trends, or key points that can be derived from these results.
"""
    payload = {
        "model": "deepseek-r1:8b",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 300
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response_data = response.json()
    insight = response_data.get("generated_text", "").strip()
    return insight


# Testing

In [6]:
chat_hist = initial_setup()

In [7]:
chat_hist

[{'role': 'system',
  'content': 'You are a friendly and helpful data analyst. The user will ask you question about data analysis, be prepared.'},
 {'role': 'assistant',
  'content': "<think>\nOkay, so I'm trying to figure out how to analyze some customer data for my company. We want to understand what our customers think of us and see where we can improve or maybe expand our services. Let me break this down step by step.\n\nFirst, I need to know who the customer is. Maybe their demographics—age, gender, location. That could help segment them later. But wait, do I have access to that data? I should check if the dataset includes these fields. If not, maybe we can collect it through surveys or look at purchase history for clues about age and gender.\n\nNext, what are they buying from us? The products or services purchased would give insights into their needs. Are there common items that people buy together? Like, if a lot of customers purchase Product A along with Service B, maybe those 

In [8]:
natural_language = "What month has the highest amount of funding?"

# Get the current directory of the script
script_dir = Path.cwd()

# Navigate to the parent directory and then into the Context folder
context_dir = script_dir.parent / "Context"

# Construct the full path to the file
file_path = context_dir / "MASTER_FUNDING.txt"

# Open context file in read mode
with open(file_path, "r") as file:
    # Read the entire file content
    content = file.read()

prompt = f"""
You are an expert SQL generator. Translate the following natural language request into a valid SQL query. 
Make sure the query is properly formatted and is tailored for our database schema.
The table in the database is master_funding, it uses the following schema:
{content}

Request: {natural_language}

SQL Query:
"""

chat_hist.append({"role":"user","content":prompt})
payload = {
    "model": "deepseek-r1:8b",  # if needed, or remove if the API defaults to it
    "messages": chat_hist,
    "temperature": 0.0,  # Lower temperature can help ensure deterministic output
    "stream":False
}
response = requests.post(OLLAMA_API_URL, json=payload).json()

In [12]:
chat_hist.append({'role':'assistant',
                  'content':response['message']['content']})

In [13]:
chat_hist

[{'role': 'system',
  'content': 'You are a friendly and helpful data analyst. The user will ask you question about data analysis, be prepared.'},
 {'role': 'assistant',
  'content': "<think>\nOkay, so I'm trying to figure out how to analyze some customer data for my company. We want to understand what our customers think of us and see where we can improve or maybe expand our services. Let me break this down step by step.\n\nFirst, I need to know who the customer is. Maybe their demographics—age, gender, location. That could help segment them later. But wait, do I have access to that data? I should check if the dataset includes these fields. If not, maybe we can collect it through surveys or look at purchase history for clues about age and gender.\n\nNext, what are they buying from us? The products or services purchased would give insights into their needs. Are there common items that people buy together? Like, if a lot of customers purchase Product A along with Service B, maybe those 

In [ ]:

# The structure may vary based on the API; assume generated text is in this field:
sql_query = response_data.get("generated_text", "").strip()